In [ ]:
import os
import math
import csv

import h5py
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
from celluloid import Camera
from matplotlib import cm
from matplotlib.colors import ListedColormap
import scipy.special as special
from scipy.stats import poisson, uniform, norm
import matplotlib.gridspec as gridspec
from matplotlib import colors
from matplotlib.colors import LogNorm
from tqdm import tqdm
from mpl_axes_aligner import align
from multiprocessing import Pool, cpu_count
from functools import partial
from scipy.signal import savgol_filter

import wf_func as wff

spe_pre = wff.read_model('spe.h5', 1)
gmu = 160.
gsigma = 40.
std = 1.

import matplotlib
matplotlib.use('Agg')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rcParams['font.size'] = 20
%matplotlib inline

rng = default_rng()

# toy 模拟分析 bias

In [ ]:
# spemode = 'spe'
# spemode = 'delta'
spemode = 'flat'
prior = False
plot = False

n = 1000
gmu = 160.
gsigma = gmu / 4
window = 3
npe = 2
assert window > npe

def spe(t, mode='spe'):
    if mode == 'spe':
        return wff.spe((t + np.abs(t)) / 2, p[0], p[1], p[2])
    elif mode == 'delta':
        return np.where(t == 0, gmu, 0)
    elif mode == 'flat':
        return np.ones_like(t) / window * gmu

noi = False
if noi:
    std = 1
else:
    std = 1e-4
np.random.seed(0)
wdtp = np.dtype([('No', np.uint32), ('Waveform', np.float, window)])
waves = np.empty(n).astype(wdtp)

tlist = np.arange(npe)

p = spe_pre[0]['parameters']
sams = [np.vstack((tlist, wff.charge(npe, gmu=gmu, gsigma=gsigma, thres=0))).T for i in range(n)]
pan = np.arange(window)
for i in tqdm(range(n)):
#     wave = np.sum([np.where(pan >= sams[i][j, 0], spe(pan - sams[i][j, 0], mode=spemode) * sams[i][j, 1] / gmu, 0) for j in range(len(sams[i]))], axis=0)
    wave = np.sum([spe(pan - sams[i][j, 0], mode=spemode) * sams[i][j, 1] / gmu for j in range(len(sams[i]))], axis=0)
    if noi:
        wave = wave + np.random.normal(0, std, size=window)
    waves[i]['Waveform'] = wave
waves['No'] = np.arange(n).astype(np.uint32)
sdtp = np.dtype([('No', np.uint32), ('HitPosInWindow', np.float64), ('Charge', np.float64)])
pelist = np.empty(sum([len(sams[i]) for i in range(n)])).astype(sdtp)
pelist['No'] = np.repeat(np.arange(n), [len(sams[i]) for i in range(n)]).astype(np.uint32)
pelist['HitPosInWindow'] = np.hstack([sams[i][:, 0] for i in range(n)])
pelist['Charge'] = np.hstack([sams[i][:, 1] for i in range(n)])

In [ ]:
mu = np.empty(n)
for i in tqdm(range(n)):
    wave = waves[i]['Waveform'].astype(np.float64)[:window] * spe_pre[0]['epulse']
    truth = pelist[pelist['No'] == waves[i]['No']]
    tlist = np.unique(truth['HitPosInWindow'])

    t_auto = np.arange(window)[:, None] - tlist
    A = spe(t_auto, spemode)
    mu_t = npe
    factor = np.sqrt(np.diag(np.matmul(A.T, A)))
    A = A / factor
    la = mu_t * np.ones_like(tlist) / len(tlist)
    
    xmmse, xmmse_star, psy_star, nu_star, nu_star_bk, T_star, d_max_i, num_i = wff.fbmpr_fxn_reduced(wave, A, std ** 2, (gsigma * factor / gmu) ** 2, factor, len(la), la, stop=5, truth=truth, i=i, left=0, right=window, tlist=tlist, gmu=gmu, para=spe_pre[0]['parameters'], prior=prior, plot=plot)
    
    c_star = np.zeros_like(xmmse_star).astype(int)
    for k in range(len(T_star)):
        t, c = np.unique(T_star[k][xmmse_star[k][T_star[k]] > 0], return_counts=True)
        c_star[k, t] = c

    mu[i] = np.average(c_star.sum(axis=1), weights=psy_star)

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(mu, bins=100, histtype='step', label='N')
ax.set_xlabel('N')
ax.set_ylabel('count')
ax.grid()
ax.legend(title='bias = {:.2%}'.format((mu.mean() - npe) / npe))
plt.show()

In [ ]:
np.random.seed(0)
for i in range(10):
    wave = waves[i]['Waveform'].astype(np.float64)[:window] * spe_pre[0]['epulse']
    truth = pelist[pelist['No'] == waves[i]['No']]
    tlist = np.unique(truth['HitPosInWindow'])

    t_auto = np.arange(window)[:, None] - tlist
    A = spe(t_auto, spemode)
    mu_t = npe
    factor = np.sqrt(np.diag(np.matmul(A.T, A)))
    A = A / factor
    la = mu_t * np.ones_like(tlist) / len(tlist)
    
    pmin = max(1, int(npe - np.sqrt(npe)))
    pmax = int(npe + 2 * np.sqrt(npe)) + 1
    pmax = npe + 1
    samnum = 1000
    nu = np.empty((pmax - pmin, samnum))
    for j in range(pmin, pmax):
        for k in range(samnum):
            ind = rng.choice(tlist, size=j, replace=False)
#             ind = np.random.choice(tlist, size=j)
            nx = np.sum(tlist[:, None] == ind, axis=1)
            nu[j - pmin][k] = wff.nu_direct(wave, A, nx, factor, (gsigma * factor / gmu) ** 2, std ** 2, la, prior=prior)
    M, N = A.shape
    p = 1 - poisson.pmf(0, la).mean()
    sig2w = std ** 2
    sig2s = (gsigma * factor / gmu) ** 2
    nu_true_mean = -M / 2 - M / 2 * np.log(sig2w) - p * N / 2 * np.log(sig2s / sig2w + 1) - M / 2 * np.log(2 * np.pi) + N * np.log(1 - p) + p * N * np.log(p / (1 - p))
    nu_true_stdv = np.sqrt(M / 2 + N * p * (1 - p) * (np.log(p / (1 - p)) - np.log(sig2s / sig2w + 1) / 2) ** 2)
    
    plt.close()
    fig = plt.figure(figsize=(16, 6))
    # fig.tight_layout()
    gs = gridspec.GridSpec(1, 2, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
    ax = fig.add_subplot(gs[0, 0])
    boxdict = ax.boxplot(nu.T, sym='', positions=np.arange(pmin, pmax), patch_artist=True)
    med = np.array([boxdict['whiskers'][ii].get_xydata()[:, 1] for ii in range(pmax - pmin)])
    ax.scatter(np.arange(pmin, pmax), nu.max(axis=1), s=-nu.max(axis=1), marker='o', facecolors='none', edgecolors='r')
#     ax.vlines(npe, med.min(), med.max(), color='b')
#     ax.hlines(nu_true_mean, pmin, pmax - 1, color='r')
#     ax.hlines(nu_true_mean + nu_true_stdv, pmin, pmax - 1, color='r', linestyles='--')
#     ax.hlines(nu_true_mean - nu_true_stdv, pmin, pmax - 1, color='r', linestyles='--')
    ax.set_xlabel('N')
    ax.set_ylabel('nu')
    ax.grid()
    
    ax = fig.add_subplot(gs[0, 1])
    ax.plot(np.arange(pmin, pmax)[1:], np.diff(nu.mean(axis=1)), label='mean diff')
    ax.plot(np.arange(pmin, pmax)[1:], np.diff(nu.max(axis=1)), label='max diff')
    ax.hlines(0, pmin + 1, pmax - 1, label='0', color='g')
    ax.hlines(np.log(1e-4), pmin + 1, pmax - 1, label='1e-4', color='r')
    ax.grid()
    ax.legend(title='N = {}'.format(npe))
    ax.set_xlabel('N')
    ax.set_ylabel('diff nu')
    plt.show()

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
camera = Camera(fig)
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
arg = np.argwhere(mu >= pmax).flatten()[:10]
for i in tqdm(arg):
    ax.plot(waves[i]['Waveform'])
    truth = pelist[pelist['No'] == waves[i]['No']]
    ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'], 'r')
    camera.snap()
ax.set_xlabel('Time')
ax.set_ylabel('Voltage')
ax.set_xlim(-1, 50)
ax.grid()
plt.show()
animation = camera.animate(interval=1000)
animation.save('animation.gif')
plt.close(fig)

# 使用单 bin 时间窗计算 $\mu$ 后验分布的 bias

In [ ]:
def getm(i, p):
    N = 10000
    bias = np.empty(N)
    np.random.seed(i)
    Mu = 10
    gmu = 160
    gsigma = 40
    npan = np.arange(1, 50)
    for i in range(N):
        n = poisson.ppf(1 - uniform.rvs(scale=1-poisson.cdf(0, Mu), size=1), Mu).astype(int)
        cha = norm.ppf(1 - uniform.rvs(scale=1-norm.cdf(0, loc=gmu, scale=gsigma), size=n), loc=gmu, scale=gsigma)
        cha = cha.sum()
        if p:
            weight = poisson.pmf(npan, Mu) * norm.pdf(cha, loc=gmu * npan, scale=gsigma * np.sqrt(npan))
        else:
            weight = norm.pdf(cha, loc=gmu * npan, scale=gsigma * np.sqrt(npan))
        weight = weight / weight.sum()
        bias[i] = np.sum(npan * weight) - n
    m = bias.mean()
    return m

In [ ]:
slices = np.arange(1000).T.tolist()
with Pool(100) as pool:
    resultpoisson = np.hstack([pool.starmap(getm, zip(slices, [True] * len(slices)))])
with Pool(100) as pool:
    resultflat = np.hstack([pool.starmap(getm, zip(slices, [False] * len(slices)))])

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(resultpoisson, bins=20, histtype='step', label='poisson')
ax.hist(resultflat, bins=20, histtype='step', label='flat')
ax.grid()
ax.legend(title='poisson mean = {:.2e}\nflat mean = {:.2e}'.format(resultpoisson.mean(), resultflat.mean()), loc='upper center')
ax.set_xlabel('bias')
ax.set_ylabel('count')
fig.savefig('note/onebinbias.png')
plt.show()

# 评估 DAQ 取整对 $\sigma_w$ 大小的影响

In [ ]:
window = 100
pan = np.arange(window)
charge = wff.charge(N, gmu=gmu, gsigma=gsigma, thres=0)
spetru = [wff.spe(pan, tau=p[0], sigma=p[1], A=p[2]) * charge[i] for i in range(N)]
spe = [np.around(spetru[i] + np.random.normal(0, std, size=window)) for i in range(N)]
diff = np.stack([spe[i] - spetru[i] for i in range(N)]).flatten()

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(-5, 5, 0.1)
ax.hist(diff, bins=200, alpha=0.5, histtype='step', density=True)
ax.plot(t, norm.pdf(t, loc=0, scale=std), c='k')
plt.show()

# $\mu$ 的 bias 随着 bin width 的变化

In [ ]:
with open('rc.csv') as f:
    f_csv = csv.reader(f, delimiter=' ')
    Tau = next(f_csv)
    Tau = [float(i) for i in Tau]
    Sigma = next(f_csv)
    Sigma = [float(i) for i in Sigma]

filelist = os.listdir('result/lucyddm/solu')
filelist = [f for f in filelist if f[0] != '.' and os.path.splitext(f)[-1] == '.h5']
numbers = [[float(i) for i in f[:-3].split('-')] for f in filelist]
stype = np.dtype([('mu', np.float64), ('tau', np.float64), ('sigma', np.float64), ('n', np.uint), ('stdmutru', np.float64), ('stdmuint', np.float64), ('stdmupe', np.float64), ('stdmumax', np.float64), ('stdmu', np.float64), ('biasmuint', np.float64), ('biasmupe', np.float64), ('biasmumax', np.float64), ('biasmu', np.float64), ('N', np.uint)])
mtsi = np.zeros(len(numbers), dtype=stype)
mtsi['mu'] = np.array([i[0] for i in numbers])
mtsi['tau'] = np.array([i[1] for i in numbers])
mtsi['sigma'] = np.array([i[2] for i in numbers])
mtsi['n'] = np.arange(len(numbers))
mtsi['stdmutru'] = np.nan
mtsi['stdmuint'] = np.nan
mtsi['stdmupe'] = np.nan
mtsi['stdmumax'] = np.nan
mtsi['stdmu'] = np.nan
mtsi['biasmuint'] = np.nan
mtsi['biasmupe'] = np.nan
mtsi['biasmumax'] = np.nan
mtsi['biasmu'] = np.nan
mtsi = np.sort(mtsi, kind='stable', order=['mu', 'tau', 'sigma'])
mts = {1:mtsi.copy(), 2:mtsi.copy(), 5:mtsi.copy(), 10:mtsi.copy()}

marker = {'int':'o', 'fbmp':'s', 'fbmpmax':'^'}
color = {'int':'g', 1:'r', 2:'b', 5:'y', 10:'m'}

for key in mts.keys():
    for i in range(len(mts[key])):
        f = filelist[mts[key][i]['n']]
        mu = mts[key][i]['mu']
        tau = mts[key][i]['tau']
        sigma = mts[key][i]['sigma']
        try:
            with h5py.File(os.path.join('result', 'fbmp-'+str(key), 'solu', f), 'r', libver='latest', swmr=True) as soluf, h5py.File(os.path.join('result', 'fbmp-'+str(key), 'dist', f), 'r', libver='latest', swmr=True) as distf, h5py.File(os.path.join('waveform', f), 'r', libver='latest', swmr=True) as wavef:
                time = soluf['starttime'][:]
                record = distf['Record'][:]
                start = wavef['SimTruth/T'][:]
                pelist = wavef['SimTriggerInfo/PEList'][:]
                waves = wavef['Readout/Waveform'][:]
                gmu = wavef['SimTriggerInfo/PEList'].attrs['gmu']
                gsigma = wavef['SimTriggerInfo/PEList'].attrs['gsigma']
                r = wavef['SimTruth/T'].attrs['r']
            mts[key][i]['N'] = len(start)
            vali = np.full(len(start['T0']), True)
            mts[key][i]['N'] = len(start)
            Chnum = len(np.unique(pelist['PMTId']))
            e_ans, i_ans = np.unique(pelist['TriggerNo'] * Chnum + pelist['PMTId'], return_index=True)
            i_ans = np.append(i_ans, len(pelist))
            pe_sum = np.array([pelist[i_ans[i]:i_ans[i+1]]['Charge'].sum() for i in range(len(e_ans))]) / gmu
            wave_sum = waves['Waveform'].sum(axis=1) / gmu
            n = np.arange(1, 1000)
            mean = np.average(n, weights=poisson.pmf(n, mu=mu))
            lognm = np.average(np.log(n), weights=poisson.pmf(n, mu=mu))
            slog = np.sqrt(np.average((np.log(n) - lognm)**2, weights=poisson.pmf(n, mu=mu)))
            mts[key][i]['stdmutru'] = slog
            slog = np.std(np.log(wave_sum[vali]), ddof=-1)
            m = np.mean(wave_sum[vali]) - mean
            mts[key][i]['stdmuint'] = slog
            mts[key][i]['biasmuint'] = m
            s = np.std(pe_sum[vali], ddof=-1)
            slog = np.std(np.log(pe_sum[vali]), ddof=-1)
            m = np.mean(pe_sum[vali]) - mean
            mts[key][i]['stdmupe'] = slog
            mts[key][i]['biasmupe'] = m
            s = np.std(time['mucharge'][vali], ddof=-1)
            slog = np.std(np.log(time['mucharge'][vali]), ddof=-1)
            m = np.mean(time['mucharge'][vali]) - mean
            mts[key][i]['stdmumax'] = slog
            mts[key][i]['biasmumax'] = m
            s = np.std(time['muwave'][vali], ddof=-1)
            slog = np.std(np.log(time['muwave'][vali]), ddof=-1)
            m = np.mean(time['muwave'][vali]) - mean
            mts[key][i]['stdmu'] = slog
            mts[key][i]['biasmu'] = m
        except:
            pass

figb = plt.figure(figsize=(len(Tau) * 5, len(Sigma) * 3))
gs = gridspec.GridSpec(1, 2, figure=figb, left=0.1, right=0.8, top=0.92, bottom=0.15, wspace=0.3, hspace=0.35)
for sigma, i in zip(Sigma, list(range(len(Sigma)))):
    for tau, j in zip(Tau, list(range(len(Tau)))):
        stdlistkey = mts[1][(mts[1]['tau'] == tau) & (mts[1]['sigma'] == sigma)]
        
        ax = figb.add_subplot(gs[i, j])
        n = np.arange(1, 1000)
        mean = np.array([np.average(n, weights=poisson.pmf(n, mu=mu)) for mu in stdlistkey['mu']])
        yerr = stdlistkey['biasmuint'] / np.sqrt(stdlistkey['N']) / mean
        ax.errorbar(stdlistkey['mu'], stdlistkey['biasmuint'] / mean, yerr=yerr, label='int', color=color['int'])
        
        for key in mts.keys():
            stdlistkey = mts[key][(mts[key]['tau'] == tau) & (mts[key]['sigma'] == sigma)]
            yerr = stdlistkey['biasmumax'] / np.sqrt(stdlistkey['N']) / mean
            ax.errorbar(stdlistkey['mu'], stdlistkey['biasmumax'] / mean, yerr=yerr, label='fbmpmax-'+str(key), color=color[key], marker=marker['fbmpmax'])
            yerr = stdlistkey['biasmu'] / np.sqrt(stdlistkey['N']) / mean
            ax.errorbar(stdlistkey['mu'], stdlistkey['biasmu'] / mean, yerr=yerr, label='fbmp-'+str(key), color=color[key], marker=marker['fbmp'])
        ax.set_xlabel(r'$N_{\mathrm{PE}}\ \mathrm{expectation}\ \mu$')
        ax.set_ylabel(r'$\frac{\Delta \mu}{\mu}$')
        ax.set_title(fr'$\tau={tau}\mathrm{{ns}},\,\sigma={sigma}\mathrm{{ns}}$')
        ax.yaxis.get_major_formatter().set_powerlimits((0, 0))
        ax.grid()
        if i == len(Sigma) - 1 and j == len(Tau) - 1:
            ax.legend(loc='upper left', bbox_to_anchor=(1., 0.9))
figb.savefig('note/vs-biasmu-bin.png')
plt.close(figb)

# 检查评估 FBMP 的先验分布对结果的影响

In [ ]:
para = '4.0-20-5'
Mu = 4
Tau = 20
Sigma = 5
wf = h5py.File('waveform/' + para + '.h5', 'r', libver='latest', swmr=True)

truth = wf['SimTruth/T'][:]
ent = wf['Readout/Waveform'][:]
pelist = wf['SimTriggerInfo/PEList'][:]

In [ ]:
def initial_params(wave, spe_pre, Tau, Sigma, gmu, Thres, p, nsp, nstd, is_t0=False, is_delta=False, n=1, nshannon=1):
#     hitt, char = wff.lucyddm(savgol_filter(wave[::nshannon], 11, 4), spe_pre['spe'][::nshannon])
    hitt, char = wff.lucyddm(wave[::nshannon], spe_pre['spe'][::nshannon])
    hitt, char = wff.clip(hitt, char, Thres)
    char = char / char.sum() * np.clip(np.abs(wave[::nshannon].sum()), 1e-6, np.inf)
    tlist = np.unique(np.clip(np.hstack(hitt[:, None] + np.arange(-nsp, nsp+1)), 0, len(wave[::nshannon]) - 1))
#     tlist = hitt

    index_prom = np.hstack([np.argwhere(savgol_filter(wave, 11 * (nshannon if nshannon % 2 == 1 else nshannon + 1), 4) > nstd * spe_pre['std']).flatten(), hitt * nshannon])
    left_wave = round(np.clip(index_prom.min() - 3 * spe_pre['mar_l'], 0, len(wave) - 1))
    right_wave = round(np.clip(index_prom.max() + 3 * spe_pre['mar_r'], 0, len(wave) - 1))
    wave = wave[left_wave:right_wave]

    npe_init = np.zeros(len(tlist))
    npe_init[np.isin(tlist, hitt)] = char / gmu
    npe_init = np.repeat(npe_init, n) / n
    tlist = np.unique(np.sort(np.hstack(tlist[:, None] + np.linspace(0, 1, n, endpoint=False)) - (n // 2) / n))
    if len(tlist) != 1:
        assert abs(np.diff(tlist).min() - 1 / n) < 1e-3, 'tlist anomalous'
    t_auto = (np.arange(left_wave, right_wave) / nshannon)[:, None] - tlist
    A = p[2] * np.exp(-1 / 2 * (np.log((t_auto + np.abs(t_auto)) / p[0] / 2) / p[1]) ** 2)

    t0_init = None
    t0_init_delta = None
    if is_t0:
        t0_init, t0_init_delta = wff.likelihoodt0(hitt=hitt, char=char, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge', is_delta=is_delta)
    return A, wave, tlist, t0_init, t0_init_delta, npe_init, left_wave, right_wave

In [ ]:
Thres = {'mcmc':std / gsigma, 'lucyddm':0, 'fbmp':1e-6}
nsp = 1
nstd = 3
n = 1000
score = np.empty(len(ent[:n]))

for i in tqdm(range(n)):
    truth = pelist[pelist['TriggerNo'] == ent[i]['TriggerNo']]
    trutlist = truth['HitPosInWindow']
    cid = ent[i]['ChannelID']
    wave = ent[i]['Waveform'].astype(np.float64) * spe_pre[cid]['epulse']
    n = 5
    A, wave_r, tlist, t0_t, t0_delta, cha, left_wave, right_wave = initial_params(wave[::wff.nshannon], spe_pre[ent[i]['ChannelID']], Tau, Sigma, gmu, Thres['lucyddm'], p, nsp, nstd, is_t0=True, is_delta=False, n=n, nshannon=1)
    mu_t = abs(wave_r.sum() / gmu)
    la = cha / cha.sum() * mu_t
    vali = np.sum((trutlist[:, None] > tlist - 1 / 2 / n) & (trutlist[:, None] < tlist + 1 / 2 / n), axis=1) > 0
    score[i] = vali.sum() / len(vali)

In [ ]:
plt.close()
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(score, bins=20, histtype='step')
ax.grid()
ax.set_xlabel('fraction')
ax.set_ylabel('count')
plt.show()